In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')
drive='/content/drive/MyDrive/Alex_weekly_reports'

Mounted at /content/drive


In [2]:
df = pd.read_csv(drive+'/merged.csv')

In [3]:
pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 5.5 MB/s eta 0:00:00


In [4]:
pip install node2vec

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.2 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.1
    Uninstalling networkx-3.1:
      Successfully uninstalled networkx-3.1


In [5]:
import networkx as nx
import re, itertools
import string
import emoji

In [6]:
df = df[df['mainContent'].notna()]
df['mainContent'] = df['mainContent'].apply(lambda s: emoji.replace_emoji(s, ''))
df['mainContent'] = df['mainContent'].str.replace('[^a-zA-Z]', ' ')

<ipython-input-6-b066d88655c7>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['mainContent'] = df['mainContent'].str.replace('[^a-zA-Z]', ' ')


In [7]:
word_dictionary = {}

In [8]:
combined_list = []
for descriptions in df['mainContent']:
    # Remove the leading spaces and newline character
    descriptions = descriptions.strip()

    # Convert the characters in line to
    # lowercase to avoid case mismatch
    descriptions = descriptions.lower()

    # Split the line into words
    words = descriptions.split(" ")
    words = list(filter(None, words))
    combined_list.append(words)

In [9]:
counter = 20
combined_list = [[i for i in lists if lists.count(i) < counter] for lists in combined_list]
print(combined_list)

[['opera', 'for', 'android', 'is', 'a', 'fast', 'all', 'around', 'browser', 'for', 'users', 'who', 'value', 'privacy', 'customization', 'and', 'sleek', 'design', 'browse', 'the', 'web', 'with', 'security', 'features', 'such', 'as', 'a', 'built', 'in', 'vpn', 'and', 'ad', 'blocker', 'while', 'enjoying', 'the', 'easy', 'to', 'use', 'interface', 'vpn', 'the', 'first', 'browser', 'which', 'offers', 'free', 'built', 'in', 'vpn', 'ad', 'blocker', 'get', 'rid', 'of', 'thousands', 'of', 'intrusive', 'ads', 'and', 'speed', 'up', 'your', 'browsing', 'private', 'browsing', 'easily', 'switch', 'between', 'normal', 'and', 'private', 'tabs', 'to', 'surf', 'the', 'web', 'without', 'leaving', 'a', 'trace', 'password', 'manager', 'opt', 'in', 'for', 'automatic', 'saving', 'of', 'passwords', 'and', 'safely', 'autofill', 'your', 'payment', 'information', 'for', 'secure', 'and', 'private', 'online', 'shopping', 'night', 'mode', 'fully', 'configurable', 'mode', 'with', 'adjustable', 'dimming', 'color', 'te

In [10]:
stopwords = ['for', 'is', 'a', 'all', 'and', 'the', 'with', 'such',
             'as', 'in', 'while', 'to', 'which', 'of', 'your', 's',
             'it', 'an', 'that', 'their', 'this', 't', 'from', 'or', 'www',
             're', 'also', 'by', 'non', 'every', 'ever', 'most', 'at', 'any',
             'll', 'at', 'if', 'on', 'so', 'no', 'https', 'about', 'la', 'http',
             'org', 'tor', 'c', 'com', 'not', 'twinme', 'thus',
             'those', 'both', 'just', 'those', 'g', 'etc', 'd', 'b', 'our', 'wi',
             'fi', 'been', 'pro', 'de', 'r', 'then', 'href', 'url', 'q', 'dhttp',
             'sa', 'dd', 'usg', 'm', 'u', 'v', 'too', 'x', 'h', 'ui', 'often', 'n',
             'ou', 'un', 'e', 'les', 'l', 'et', 'des', 'en', 'vous', 'le', 'y', 'z',
             'tuo', 'di', 'i', 'o', 'my', 'k', 've', 'ly', 'cor', 'w', 'might', 'goo',
             'gl', 'dhttps', 'aes', 'j', 'lg', 'htc', 'thinkyeah', 'du', 'us']

combined_list = [[i for i in lists if i not in stopwords] for lists in combined_list]

In [11]:
print(sum([len(i) for i in combined_list]))


34746


In [12]:
def get_previous_and_current_words_from_list(list_of_lists):
    big_list = []

    for inner_list in list_of_lists:
        word_pairs = []
        previous_word = None

        for current_word in inner_list:
            if previous_word is not None:
                word_pairs.append((previous_word + ' ' + current_word))
            previous_word = current_word

        big_list.append(word_pairs)

    return big_list

In [13]:
def count_words_in_list_of_lists(list_of_lists):
    result_dicts = []

    for inner_list in list_of_lists:
        word_count = {}
        for word in inner_list:
            word_count[word] = word_count.get(word, 0) + 1
        result_dicts.append(word_count)

    return result_dicts


In [14]:
result_list = get_previous_and_current_words_from_list(combined_list)
result_dicts = count_words_in_list_of_lists(result_list)


In [15]:
#for i in combined_list:
  #print(i)

In [16]:
#for i in result_list:
  #print(i)

In [17]:
len(result_dicts)

213

In [18]:
G_list = []
for pairs in result_dicts:
  G = nx.Graph()
  for word, weight in pairs.items():
    seperate = word.split(' ')
    G.add_node(seperate[0])
    G.add_node(seperate[1])
    G.add_edge(seperate[0], seperate[1], weight = weight)
  G_list.append(G)


In [19]:
print(G_list)

[<networkx.classes.graph.Graph object at 0x7c2ad54e1cf0>, <networkx.classes.graph.Graph object at 0x7c2ad54e1030>, <networkx.classes.graph.Graph object at 0x7c2ad54e2f20>, <networkx.classes.graph.Graph object at 0x7c2ad54e3a90>, <networkx.classes.graph.Graph object at 0x7c2ad54e0d60>, <networkx.classes.graph.Graph object at 0x7c2ad54e0d00>, <networkx.classes.graph.Graph object at 0x7c2ad54e0ca0>, <networkx.classes.graph.Graph object at 0x7c2ad54e0c40>, <networkx.classes.graph.Graph object at 0x7c2ad54e1870>, <networkx.classes.graph.Graph object at 0x7c2ad54e18d0>, <networkx.classes.graph.Graph object at 0x7c2ad54e1930>, <networkx.classes.graph.Graph object at 0x7c2ad54e1990>, <networkx.classes.graph.Graph object at 0x7c2ad54e19f0>, <networkx.classes.graph.Graph object at 0x7c2ad54e1fc0>, <networkx.classes.graph.Graph object at 0x7c2ad54e3790>, <networkx.classes.graph.Graph object at 0x7c2ad54e3700>, <networkx.classes.graph.Graph object at 0x7c2ad54e36a0>, <networkx.classes.graph.Graph 

In [20]:
from node2vec import Node2Vec

In [21]:
graph_embeddings = []
g_emb=np.zeros(128)
for G in G_list:
    node2vec = Node2Vec(G, dimensions=128, walk_length=10, num_walks=50, p=1, q=1, workers=4)
    model = node2vec.fit()
    print(model)
    node_embeddings = {node: model.wv[node] for node in G.nodes()}
    for j in (node_embeddings.values()):
        g_emb+=j

    g_emb=g_emb/len(node_embeddings)
    graph_embeddings.append(g_emb)

Computing transition probabilities:   0%|          | 0/145 [00:00<?, ?it/s]

Word2Vec<vocab=145, vector_size=128, alpha=0.025>


Computing transition probabilities:   0%|          | 0/244 [00:00<?, ?it/s]

Word2Vec<vocab=244, vector_size=128, alpha=0.025>


Computing transition probabilities:   0%|          | 0/191 [00:00<?, ?it/s]

...

Word2Vec<vocab=64, vector_size=128, alpha=0.025>


Computing transition probabilities:   0%|          | 0/14 [00:00<?, ?it/s]

Word2Vec<vocab=14, vector_size=128, alpha=0.025>


Computing transition probabilities:   0%|          | 0/46 [00:00<?, ?it/s]

Word2Vec<vocab=46, vector_size=128, alpha=0.025>


In [22]:
print(len(graph_embeddings))

213


In [23]:
#Getting the Vectors and Splitting the Data
from sklearn.model_selection import train_test_split
# Convert the list of feature vectors into a numpy array
vectors = graph_embeddings
#print(vectors[200])
print(len(vectors))
labels = df['isVault']
#print(labels[213])
print(len(labels))
X_train, X_test, y_train, y_test = train_test_split(vectors, labels, test_size = 0.4, random_state=60)

213
213


In [24]:
print(labels)

0      0
1      0
2      1
3      1
4      0
      ..
222    1
223    1
224    1
225    0
226    1
Name: isVault, Length: 213, dtype: int64


In [25]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()

logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

print(y_test)
print(y_pred)

#Logistic Regression Accuracy Score
from sklearn import metrics
print('Accuracy Score:')
logreg = metrics.accuracy_score(y_test, y_pred)
print(logreg)
print(metrics.classification_report( y_test,y_pred, labels=None))

68     0
211    1
214    1
195    0
162    0
      ..
175    0
53     0
2      1
133    0
121    0
Name: isVault, Length: 86, dtype: int64
[0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 1 0 1 1 1 0 0 1 0 0 1 0 0 0 0 0 1 0 1
 0 1 1 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 1 1]
Accuracy Score:
0.6046511627906976
              precision    recall  f1-score   support

           0       0.70      0.75      0.72        59
           1       0.35      0.30      0.32        27

    accuracy                           0.60        86
   macro avg       0.52      0.52      0.52        86
weighted avg       0.59      0.60      0.60        86



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [26]:
#SVM
from sklearn import svm

svc = svm.SVC(kernel = 'linear')

svc.fit(X_train, y_train)

y_pred = svc.predict(X_test)

print(y_test)
print(y_pred)

#SVM Accuracy Score
from sklearn import metrics
print('Accuracy Score:')
svm = metrics.accuracy_score(y_test, y_pred)
print(svm)
print(metrics.classification_report( y_test,y_pred, labels=None))

68     0
211    1
214    1
195    0
162    0
      ..
175    0
53     0
2      1
133    0
121    0
Name: isVault, Length: 86, dtype: int64
[0 0 0 1 0 0 1 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 1 0 1 1 1
 0 1 1 1 1 0 0 1 0 0 1 1 0 0 1 0 0 0 1 1 1 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0
 0 0 1 1 0 1 0 0 0 1 1 1]
Accuracy Score:
0.5697674418604651
              precision    recall  f1-score   support

           0       0.70      0.64      0.67        59
           1       0.34      0.41      0.37        27

    accuracy                           0.57        86
   macro avg       0.52      0.53      0.52        86
weighted avg       0.59      0.57      0.58        86



In [27]:
#Naive Bayes
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()

gnb.fit(X_train, y_train)

y_pred = gnb.predict(X_test)

print(y_test)
print(y_pred)

#Naive Bayes Accuracy Score
from sklearn import metrics
print('Accuracy Score:')
nbayes = metrics.accuracy_score(y_test, y_pred)
print(nbayes)
print(metrics.classification_report( y_test,y_pred, labels=None))

68     0
211    1
214    1
195    0
162    0
      ..
175    0
53     0
2      1
133    0
121    0
Name: isVault, Length: 86, dtype: int64
[0 0 1 0 0 0 1 0 0 0 0 0 1 0 1 1 0 0 0 0 1 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 1 0 1 0 0 0 1 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0]
Accuracy Score:
0.6976744186046512
              precision    recall  f1-score   support

           0       0.74      0.86      0.80        59
           1       0.53      0.33      0.41        27

    accuracy                           0.70        86
   macro avg       0.63      0.60      0.60        86
weighted avg       0.67      0.70      0.68        86



In [28]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()

rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

print(y_test)
print(y_pred)

#Random Forest Accuracy Score
from sklearn import metrics
print('Accuracy Score:')
rforest = metrics.accuracy_score(y_test, y_pred)
print(rforest)
print(metrics.classification_report( y_test,y_pred, labels=None))

68     0
211    1
214    1
195    0
162    0
      ..
175    0
53     0
2      1
133    0
121    0
Name: isVault, Length: 86, dtype: int64
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0]
Accuracy Score:
0.6744186046511628
              precision    recall  f1-score   support

           0       0.68      0.98      0.81        59
           1       0.00      0.00      0.00        27

    accuracy                           0.67        86
   macro avg       0.34      0.49      0.40        86
weighted avg       0.47      0.67      0.55        86



In [29]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()

dt.fit(X_train, y_train)

y_pred = dt.predict(X_test)

print(y_test)
print(y_pred)

#Decision Tree Accuracy Score
from sklearn import metrics
print('Accuracy Score:')
dtree = metrics.accuracy_score(y_test, y_pred)
print(dtree)
print(metrics.classification_report( y_test,y_pred, labels=None))

68     0
211    1
214    1
195    0
162    0
      ..
175    0
53     0
2      1
133    0
121    0
Name: isVault, Length: 86, dtype: int64
[1 0 0 0 0 1 1 0 1 0 0 1 1 0 1 1 1 0 0 0 1 1 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0
 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 1 0 0
 0 1 0 1 0 0 0 0 0 0 0 0]
Accuracy Score:
0.5465116279069767
              precision    recall  f1-score   support

           0       0.66      0.69      0.68        59
           1       0.25      0.22      0.24        27

    accuracy                           0.55        86
   macro avg       0.46      0.46      0.46        86
weighted avg       0.53      0.55      0.54        86



In [30]:
#K-Nearest Neighbor
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()

knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

print(y_test)
print(y_pred)

#K-Nearest Neighbor Accuracy Score
from sklearn import metrics
print('Accuracy Score:')
knn = metrics.accuracy_score(y_test, y_pred)
print(knn)

68     0
211    1
214    1
195    0
162    0
      ..
175    0
53     0
2      1
133    0
121    0
Name: isVault, Length: 86, dtype: int64
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0]
Accuracy Score:
0.6627906976744186


In [31]:
index = ['Text Embedding Classification']
table = pd.DataFrame({'Logistic Regression': round(logreg * 100, 2), 'Support Vector Machine': round(svm * 100, 2), 'Naive Bayes': round(nbayes * 100, 2), 'Random Forest': round(rforest * 100, 2), 'Decision Tree': round(dtree * 100, 2), 'K-Nearest Neighbor': round(knn * 100, 2)}, index = index)
print(table)

                               Logistic Regression  Support Vector Machine  \
Text Embedding Classification                60.47                   56.98   

                               Naive Bayes  Random Forest  Decision Tree  \
Text Embedding Classification        69.77          67.44          54.65   

                               K-Nearest Neighbor  
Text Embedding Classification               66.28  
